# Stata Navigation Project

## Project Outline

* [Load and Test Pretrained Places365 Model](#load-and-test-pretrained-places365-model)
* Load and Process Images of Stata
* Retrain Places365 Model
* Test Results

## Load and Test Pretrained Places365 Model

In [2]:
#imports
import torch
from torchvision import models, transforms
from PIL import Image
import urllib

In [5]:
#Load previously trained places365 model
model = models.resnet50(num_classes=365)
model_file = 'resnet50_places365.pth.tar'
checkpoint = torch.load(model_file, map_location='cpu')
state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
model.load_state_dict(state_dict)

model.eval()  # Set the model to evaluation mode

# Check if a gpu is available and move the model to the appropriate device
if torch.cuda.is_available():
    model = model.to('cuda')
else:
    model = model.to('cpu')


# Define an image preprocessing function
# The model expects images of size 224x224 and normalized with ImageNet statistics
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet means
        std=[0.229, 0.224, 0.225]    # ImageNet stds
    )
])

# Load the class labels
file_name = 'categories_places365.txt'
classes = list()
with open(file_name) as class_file:
    for line in class_file:
        classes.append(line.strip().split(' ')[0][3:])
classes = tuple(classes)


# Load an image from a URL image address
image_url = 'https://images.unsplash.com/photo-1597290282695-edc43d0e7129?fm=jpg&q=60&w=3000&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxzZWFyY2h8NHx8YmFyfGVufDB8fDB8fHww'  # replace with your image URL
img = Image.open(urllib.request.urlopen(image_url)).convert('RGB')

# Preprocess the image
input_img = transform(img).unsqueeze(0)  # add batch dimension

# Forward pass
with torch.no_grad():
    logits = model(input_img)
    probs = torch.nn.functional.softmax(logits, dim=1)

# Get the top-5 predictions
top5_probs, top5_idxs = probs.topk(5)

print("Top 5 predicted places:")
for prob, idx in zip(top5_probs[0], top5_idxs[0]):
    print(f"{classes[idx.item()]} ({prob.item():.4f})")

Top 5 predicted places:
bar (0.7915)
pharmacy (0.1112)
beer_hall (0.0362)
sushi_bar (0.0170)
drugstore (0.0123)
